In [9]:
%matplotlib notebook

import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import queue

In [10]:
# constants

GRID_RESOLUTION = 10**4 # higher res means smaller cells
CLUSTERED_DIAMETER = 100 # ZxZ clusters

MIN_LAT_IN_DATASET = 16.4078522359116
MIN_LNG_IN_DATASET = 49.112730179544
GRID_CORNER_PADDING = 100
CORNER_LAT_POS = math.floor(MIN_LAT_IN_DATASET * GRID_RESOLUTION) - GRID_CORNER_PADDING
CORNER_LNG_POS = math.floor(MIN_LNG_IN_DATASET * GRID_RESOLUTION) - GRID_CORNER_PADDING
GRID_SIZE = 3500

NOCLUSTERS = int(GRID_SIZE / CLUSTERED_DIAMETER)

PERCENT_ITEMS = 0.25

In [11]:
def linspaceExtend(route, num):
    newpoints = []
    for i in range(len(route)-1):
        point1 = route[i]
        point2 = route[i+1]
        
        lat1 = point1['lat']
        lat2 = point2['lat']
        lng1 = point1['lng']
        lng2 = point2['lng']
        
        lats = np.linspace(lat1, lat2, num=num)
        lngs = np.linspace(lng1, lng2, num=num)
        
        newpoints = newpoints + [{"lat":a, "lng":b} for (a,b) in zip(lats, lngs)]
    return newpoints

In [12]:
# read planned routes from file

planned_routes = []
with open('data/brno_cyclo_route_general_plan.csv', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=',')
    for row in csvreader:
        route_str = row['wkt']
        route_str = route_str.replace("LINESTRING (", "").replace(")", "")
        
        points = route_str.split(',')
        route = []
        for point in points:
            coords = point.split(' ')
            lat = float(coords[0])
            lng = float(coords[1])
            route.append({"lat":lat, "lng":lng})
            
        planned_routes.append(route)
        
#for route in planned_routes:
#   planned_points = planned_points + linspaceExtend(route, 10)
# planned_routes = list(map(lambda r: linspaceExtend(r, 10), planned_routes))

planned_points = [points for route in planned_routes for points in route]
    
# lats = list(map(lambda p: p['lat'], planned_points))
# lngs = list(map(lambda p: p['lng'], planned_points))

In [ ]:
# grid matrix

class Grid:
    def __init__(self):
        #self.grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.grid = np.zeros((GRID_SIZE, GRID_SIZE))
        self.walks = np.zeros((GRID_SIZE, GRID_SIZE))        
        self.zones = np.zeros((NOCLUSTERS, NOCLUSTERS))
        self.items = []
    
    @staticmethod
    def latToCellIndex(lat):
        return math.floor(lat*GRID_RESOLUTION) - CORNER_LAT_POS
    
    @staticmethod
    def lngToCellIndex(lng):
        return math.floor(lng*GRID_RESOLUTION) - CORNER_LNG_POS
    
    @staticmethod
    def cellIndexToLat(cell):
        return (cell + CORNER_LAT_POS + 0.5) / GRID_RESOLUTION
    
    @staticmethod
    def cellIndexToLng(cell):
        return (cell + CORNER_LNG_POS + 0.5) / GRID_RESOLUTION 
    
    @staticmethod
    def getClusterCenters():
        centers = []
        for i in range(NOCLUSTERS):
            for j in range(NOCLUSTERS):
                centers.append([i*NOCLUSTERS + j, Grid.cellIndexToLat(i), Grid.cellIndexToLng(j)])
        return centers
    
    def pointToCell(self, point):
        lat = point['lat']
        lng = point['lng']
        return self.latToCellIndex(lat), self.lngToCellIndex(lng)
    
    def addAllPoints(self, points):
        for point in points:
            x,y = self.pointToCell(point)
            self.grid[x][y] = 1
            
        for i in range(len(points)-1):
            p1 = points[i]
            p2 = points[i+1]
            
            x1,y1 = self.pointToCell(p1)
            x2,y2 = self.pointToCell(p2)
            
            steps = max(x2-x1, y2-y1) * 20
            
            tx, ty = x1, y1
            for j in range(steps):
                tx = tx + (x2-x1)/steps
                ty = ty + (y2-y1)/steps
                self.grid[math.floor(tx)][math.floor(ty)] = 1
                
    def addWalkPoints(self, walks):
        for walk in walks:
            points = walk['points']
            score = walk['score']
            
            for point in points:
                x,y = self.pointToCell(point)
                self.walks[x][y] = self.walks[x][y] + score

            for i in range(len(points)-1):
                p1 = points[i]
                p2 = points[i+1]

                x1,y1 = self.pointToCell(p1)
                x2,y2 = self.pointToCell(p2)

                steps = max(x2-x1, y2-y1) * 20

                tx, ty = x1, y1
                for j in range(steps):
                    tx = tx + (x2-x1)/steps
                    ty = ty + (y2-y1)/steps
                    self.grid[math.floor(tx)][math.floor(ty)] = self.grid[math.floor(tx)][math.floor(ty)] + score
        
    def splitToItems(self):
        items = []
        
        # horizontal items
        for i in range(GRID_SIZE):
            item = []
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 1:
                    item.append([i,j])
                else:
                    if len(item) > 1:
                        items.append(item)
                        for [a,b] in item:
                            self.grid[a][b] = 2
                        item = []
            if len(item) > 0:
                items.append(item)
                for [a,b] in item:
                    self.grid[a][b] = 2
                item = []
                
        # vertical items
        for j in range(GRID_SIZE):
            item = []
            for i in range(GRID_SIZE):
                if self.grid[i][j] == 1:
                    item.append([i,j])
                else:
                    if len(item) > 0:
                        items.append(item)
                        for [a,b] in item:
                            self.grid[a][b] = 2
                        item = []
            if len(item) > 0:
                items.append(item)
                for [a,b] in item:
                    self.grid[a][b] = 2
                item = []
        
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 2:
                    self.grid[i][j] = 1
        
        self.items = items
        print(len(items))
        
    def computeOverlapping(self):
        totalitems = len(self.items)
        q = queue.PriorityQueue(maxsize=int(totalitems * PERCENT_ITEMS))
        
        for item in self.items:
            itemscore = 0
            for [x,y] in item:
                itemscore = itemscore + self.walks[x,y]
            itemscore = itemscore / len(item) #### is this good ??????????
            q.put((-itemscore, item))
        
        for i in range(int(totalitems * PERCENT_ITEMS)):
            (_, item) = q.get()
            for [x,y] in item:
                #self.grid[x][y] = 0.5
                pass
        
# create matrix for planned points
g = Grid()

In [ ]:
# add planned routes

#g.addAllPoints(planned_points)
for route in planned_routes:
    g.addAllPoints(route)
g.splitToItems()

plt.ioff()
plt.matshow(g.grid, aspect='auto')
plt.savefig('grid_map2.png', dpi=500)

In [ ]:
clusters = Grid.getClusterCenters()
print(len(clusters))
with open("clusters.csv", 'w') as file:
    for cluster in clusters:
        file.write(str(cluster[0]) + ',' + str(cluster[1]) + ',' + str(cluster[2]) + '\n')

In [ ]:
# add walk routes
walk_routes = [] #### TODO

for walk in walk_routes:
    g.addWalkPoints(walk)

g.computeOverlapping()

plt.ioff()
plt.matshow(g.grid, aspect='auto')
plt.savefig('grid_map3.png', dpi=500)